In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

import sys
import importlib

sys.path.append(r"C:\Users\mattm\OneDrive\Desktop\Research\Projects\Triangle Lattice\Jupyter Notebooks\8Q_Triangle_Lattice_v1")

import correlation_measurements.src_correlation_measurement
importlib.reload(correlation_measurements.src_correlation_measurement)
from correlation_measurements.src_correlation_measurement import RampOscillationShotsMeasurement, generate_ramp_double_jump_correlations_filename

import src.src_current_measurement
importlib.reload(src.src_current_measurement);
from src.src_current_measurement import CurrentMeasurementCalibration, generate_current_calibration_filename, acquire_data, generate_ramp_beamsplitter_correlations_filename, generate_ramp_beamsplitter_correlations_clean_filename



In [ ]:
def calculate_current_correlation_from_counts(counts, population_average=None, basis=None, readout_pair_1=[0,1], readout_pair_2=[2,3]):
    if not abs(readout_pair_1[1] - readout_pair_1[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_1}')

    if not abs(readout_pair_2[1] - readout_pair_2[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_2}')

    if basis is None:
        basis = list(product(range(2), repeat=math.ceil(math.log2(counts.shape[0]))))

    if population_average is None:
        population_average = calculate_population_from_counts(counts, basis=basis, readout_pair_1=readout_pair_1, readout_pair_2=readout_pair_2)

    num_shots = np.sum(counts, axis=0)


    current_correlations = np.zeros(counts.shape[-1])

    for i, index_1 in enumerate(readout_pair_1):
        for j, index_2 in enumerate(readout_pair_2):

            num_shots_post_selected = 0


            outcome_count = 0
            for outcome_idx, outcome in enumerate(basis):
                if sum(outcome) == 4:
                    num_shots_post_selected += counts[outcome_idx,:]
                if outcome[index_1] == 1 and outcome[index_2] == 1:
                        outcome_count += counts[outcome_idx,:]

            outcome_average = outcome_count/num_shots_post_selected

            if (i + j) % 2 == 0:
                 current_correlations += outcome_average - population_average[i, :] * population_average[j, :]
            else:
                 current_correlations -= (outcome_average - population_average[i, :] * population_average[j, :])

    return current_correlations

def calculate_population_from_counts(counts, basis=None, readout_pair_1=[0,1], readout_pair_2=[2,3]):
    if not abs(readout_pair_1[1] - readout_pair_1[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_1}')

    if not abs(readout_pair_2[1] - readout_pair_2[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_2}')

    if basis is None:
        basis = list(product(range(2), repeat=math.ceil(math.log2(counts.shape[0]))))

    population_average = np.zeros((4, counts.shape[-1]))

    num_shots = np.sum(counts, axis=0)

    for i in range(len(readout_pair_1 + readout_pair_2)):

        num_shots_post_selected = 0

        for outcome_idx, outcome in enumerate(basis):
            if sum(outcome) == 4:
                num_shots_post_selected += counts[outcome_idx,:]
            if outcome[(readout_pair_1 + readout_pair_2)[i]] == 1:
                population_average[i, :] += counts[outcome_idx, :]

        population_average[i, :] /= num_shots_post_selected

    return population_average




In [ ]:
def plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=None, ylim=None):
   
    times = measurement.get_times()

    plt.plot(times, covariance_sum, 'o--')
    plt.xlabel('Time (ns)')
    plt.ylabel('Current Correlation')

    if ylim is not None:
        plt.ylim(*ylim)

    if beamsplitter_time is not None:
        plt.axvline(beamsplitter_time, color='r', alpha=0.4, linestyle='--', label='Beamsplitter Time')

    correlator_symbol = f'$\\langle j_{{{readout_pair_1[0]+1}{readout_pair_1[1]+1}}}j_{{{readout_pair_2[0]+1}{readout_pair_2[1]+1}}}\\rangle$ '
    plt.title(f'Post-Selected Current Correlations {correlator_symbol} for 8 qubits')
    plt.show()




In [ ]:
def calculate_population_from_counts(counts, basis=None, readout_pair_1=[0,1], readout_pair_2=[2,3]):
    """
    Compute single-qubit populations for the four qubits in readout_pair_1 and readout_pair_2.
    """
    if not abs(readout_pair_1[1] - readout_pair_1[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_1}')
    if not abs(readout_pair_2[1] - readout_pair_2[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_2}')

    if basis is None:
        basis = list(product(range(2), repeat=int(math.log2(counts.shape[0]))))

    # Normalize counts to probabilities per time slice
    counts = counts / np.sum(counts, axis=0, keepdims=True)

    qubit_indices = readout_pair_1 + readout_pair_2
    population_average = np.zeros((4, counts.shape[-1]))

    # Compute <n_i> for each of the four qubits of interest
    for i, q_index in enumerate(qubit_indices):
        for outcome_idx, outcome in enumerate(basis):
            if outcome[q_index] == 1:
                population_average[i, :] += counts[outcome_idx, :]

    return population_average


def calculate_current_correlation_from_counts(counts, population_average=None, basis=None,
                                              readout_pair_1=[0,1], readout_pair_2=[2,3]):
    """
    Compute the current correlation
    O = <n1 n3> - <n1 n4> - <n2 n3> + <n2 n4>,
    using properly normalized post-selected counts.
    """

    if not abs(readout_pair_1[1] - readout_pair_1[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_1}')
    if not abs(readout_pair_2[1] - readout_pair_2[0]) == 1:
        raise ValueError(f'readout pair qubits must be sequential, given: {readout_pair_2}')

    if basis is None:
        basis = list(product(range(2), repeat=int(math.log2(counts.shape[0]))))

    # Normalize counts after post-selection → makes probabilities conditional
    counts = counts / np.sum(counts, axis=0, keepdims=True)

    # If not provided, compute populations now
    if population_average is None:
        population_average = calculate_population_from_counts(
            counts, basis=basis, readout_pair_1=readout_pair_1, readout_pair_2=readout_pair_2
        )


    # Define the four qubit indices of interest
    q1, q2 = readout_pair_1
    q3, q4 = readout_pair_2

    # Compute all pairwise <n_i n_j>
    pair_means = np.zeros((4, counts.shape[-1]))
    pair_indices = [(q1, q3), (q1, q4), (q2, q3), (q2, q4)]

    for k, (i_idx, j_idx) in enumerate(pair_indices):
        outcome_count = np.zeros(counts.shape[-1])
        for outcome_idx, outcome in enumerate(basis):
            if outcome[i_idx] == 1 and outcome[j_idx] == 1:
                outcome_count += counts[outcome_idx, :]
        pair_means[k, :] = outcome_count  # Already normalized

    # Compute <n_i> values for subtraction
    n1, n2, n3, n4 = population_average

    # Covariances
    n1n3 = pair_means[0, :] - n1 * n3
    n1n4 = pair_means[1, :] - n1 * n4
    n2n3 = pair_means[2, :] - n2 * n3
    n2n4 = pair_means[3, :] - n2 * n4

    # Combine into current correlation
    current_correlations = n1n3 - n1n4 - n2n3 + n2n4

    return current_correlations


In [ ]:
name_to_filename = {}

# 9/24/25 - 8Q
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '24', '16', '53', '27') # best
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '24', '16', '59', '42')


# 10/20/25
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '20', '13', '39', '31')
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '20', '15', '55', '45')

# clean
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '20', '13', '45', '27')
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '20', '15', '58', '19')


### J_||=2J from now on (need to actually check the coupling strength still but we moved the couplers)
# 10/22/25
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '22', '16', '51', '45')

# 10/23/25
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '23', '13', '01', '14')
# name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '23', '15', '00', '52')


name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '23', '15', '04', '27')

# 10/24/25
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '24', '14', '53', '35')

# 10/30/25
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '30', '15', '19', '34')

# 11/04/25
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '11', '04', '11', '19', '24') # high quality 1234

name_to_measurement = {}
for name in name_to_filename:
    name_to_measurement[name] = RampOscillationShotsMeasurement(name_to_filename[name])

In [ ]:
state_even = '4P8Q_1234'
state_odd = '4P8Q_1254'

measurement_even = name_to_measurement[state_even]
measurement_odd = name_to_measurement[state_odd]

In [ ]:
### plot post selected data


post_select = True
confusion_matrix_correct = True

plot_individual_terms = False

rung_to_readout_pairs = {
    2: [[0, 1], [2, 3]],
    3: [[0, 1], [4, 3]],
    4: [[0, 1], [4, 5]],
    5: [[0, 1], [6, 5]],
    6: [[0, 1], [6, 7]],
}

J_12 = 6.02*2*np.pi
rung_to_coupling = {
    2: 6.01*2*np.pi,
    3: 6.23*2*np.pi,
    4: 6.07*2*np.pi,
    5: 5.83*2*np.pi,
    6: 6.37*2*np.pi,
}

beamsplitter_offset = 5

plot_rungs = [2, 4, 6]
# plot_rungs = [2, 3, 4, 5, 6]
# plot_rungs = [3, 5]
ylim = (-0.35, 0.35)

basis = list(product([0,1], repeat=8))

correlations_data = []

for rung in plot_rungs:
    if not rung in rung_to_readout_pairs:
        continue
    readout_pair_1, readout_pair_2 = rung_to_readout_pairs[rung]

    print(f'Rung {rung}:')

    average_coupling = (J_12 + rung_to_coupling[rung])/2
    beamsplitter_time = abs((np.pi/4)/(average_coupling))*1e3 + beamsplitter_offset  # in ns
    print(f'beamsplitter time: {beamsplitter_time:.2f} ns')

    print(rung_to_readout_pairs[rung])
    if rung % 2 == 0:
        measurement = measurement_even
    else:
        measurement = measurement_odd
        
    
    ### correct and post select

    # maintain the same shape as counts, but just zero out the unwanted bitstrings
    num_particles = 4

    counts = measurement.get_counts()


   
    if confusion_matrix_correct:
        confusion_matrices = measurement.get_confusion_matrices()
        confusion_inverse_matrices = np.array([np.linalg.inv(confusion_matrix) for confusion_matrix in confusion_matrices])

        joint_confusion_inverse_matrix = confusion_inverse_matrices[0]
        for i in range(1, confusion_inverse_matrices.shape[0]):
            joint_confusion_inverse_matrix = np.kron(joint_confusion_inverse_matrix, confusion_inverse_matrices[i])


        counts_corrected = joint_confusion_inverse_matrix @ counts

    else:
        counts_corrected = counts


    counts_post_selected = np.zeros_like(counts)
    counts_corrected_post_selected = np.zeros_like(counts)

    if post_select:
        for i, outcome in enumerate(basis):
            if sum(outcome) == num_particles:
                counts_post_selected[i, :] = counts[i, :]
                counts_corrected_post_selected[i, :] = counts_corrected[i, :]
    else:
        counts_post_selected = counts
        counts_corrected_post_selected = counts_corrected

    covariance_sum = calculate_current_correlation_from_counts(counts_corrected_post_selected, population_average=None,
                                                               basis=basis, readout_pair_1=readout_pair_1, readout_pair_2=readout_pair_2)

    
    plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=beamsplitter_time, ylim=ylim)


    current_correlation_index = np.argmin(np.abs(measurement.get_times() - beamsplitter_time))
    current_correlation_value = covariance_sum[current_correlation_index]

    print(f'Current correlation at beamsplitter time for rungs ({readout_pair_1[0]+1},{readout_pair_1[1]+1}) and ({readout_pair_2[0]+1},{readout_pair_2[1]+1}): {current_correlation_value:.4f}')
    correlations_data.append(current_correlation_value)

In [ ]:
# correlations_data = [0.18, -0.08, 0.06, -0.03, 0.08] # 9/24/25

expected_correlations_5us = [0.185, -0.0787, 0.0787, -0.0519, 0.0779]
expected_beamsplitter_correlations_5us = [0.144, -0.0723, 0.002, -0.041, 0.036]

expected_correlations = [0.32, -0.19, 0.13, -0.10, 0.08]
expected_beamsplitter_correlations = [0.34, -0.147, 0.096, -0.08, 0.09]

distances = [1, 2, 3, 4, 5]
distances_data = np.array(plot_rungs) - 1

# plt.plot(distances, np.abs(expected_correlations), label='simulation')
plt.plot(distances, np.abs(expected_correlations_5us), label='simulation (5 $\\mu$s)')
plt.plot(distances, np.abs(expected_correlations), alpha=0)

# plt.plot(distances, np.abs(expected_beamsplitter_correlations), label='beamsplitter simulation', linestyle='dotted', color='green')
# plt.plot(distances, np.abs(expected_beamsplitter_correlations_5us), label='beamsplitter simulation (5 $\\mu$s)', linestyle='dotted', color='green')


plt.plot(distances_data, np.abs(correlations_data), linestyle='', marker='o', color='blue', ms=8, label='magnitude')
plt.plot(distances_data, np.abs(correlations_data), linestyle='dashed', color='blue', alpha=0.5)
plt.plot(distances_data, (correlations_data), linestyle='', marker='x', color='red', ms=8, label='data')
plt.plot(distances_data, (correlations_data), linestyle='dashed', color='red', alpha=0.5)


# plt.xlabel('rung distance')
plt.xticks(ticks=distances, labels=[f'{int(d)}' for d in distances])
plt.xlabel('distance')
plt.ylabel('Current Correlation')

plt.grid()
plt.title('Current Correlation vs Rung Distance')


plt.legend()
plt.show()

In [ ]:
plt.plot(populations_post_selected[0, :], 'o--', label=f'Q{1}')
plt.plot(populations_post_selected[1, :], 'o--', label=f'Q{2}')

plt.xlabel('Time (ns)')
plt.ylabel('Population')
plt.legend()
plt.show()

for rung in plot_rungs:
    plt.plot(populations_post_selected[rung, :], 'o--', label=f'Q{rung+1}')
    plt.plot(populations_post_selected[rung+1, :], 'o--', label=f'Q{rung+2}')

    plt.xlabel('Time (ns)')
    plt.ylabel('Population')
    plt.legend()
    plt.show()

### testing

In [ ]:
### baseline
measurement = measurement_even
counts = measurement.get_counts()
times = measurement.get_times()

basis = list(product(range(2), repeat=8))

readout_pair_1 = [0,1]
readout_pair_2 = [2,3]

current_correlation_counts = calculate_current_correlation_from_counts(counts, basis=basis, readout_pair_1=readout_pair_1, 
                                                                       readout_pair_2=readout_pair_2)

plt.plot(times, current_correlation_counts, 'o--')

In [ ]:
### testing with corrections


confusion_matrices = measurement.get_confusion_matrices()

print(confusion_matrices.shape)

confusion_inverse_matrices = np.array([np.linalg.inv(confusion_matrix) for confusion_matrix in confusion_matrices])


joint_confusion_inverse_matrix = confusion_inverse_matrices[0]
for i in range(1, confusion_inverse_matrices.shape[0]):
    joint_confusion_inverse_matrix = np.kron(joint_confusion_inverse_matrix, confusion_inverse_matrices[i])


counts_corrected = joint_confusion_inverse_matrix @ counts

current_correlation_counts_corrected = calculate_current_correlation_from_counts(counts_corrected,population_average=population_average, 
                                                                       basis=basis, readout_pair_1=readout_pair_1, readout_pair_2=readout_pair_2)


plt.plot(times, current_correlation_counts_corrected, 'o--')


In [ ]:
### post select

# maintain the same shape as counts, but just zero out the unwanted bitstrings
num_particles = 4

counts_post_selected = np.zeros_like(counts)
counts_corrected_post_selected = np.zeros_like(counts)
for i, outcome in enumerate(basis):
    if sum(outcome) == num_particles:
        counts_post_selected[i, :] = counts[i, :]
        counts_corrected_post_selected[i, :] = counts_corrected[i, :]

current_correlation_counts_post_selected = calculate_current_correlation_from_counts(counts_post_selected,population_average=None, 
                                                                       basis=basis, readout_pair_1=readout_pair_1, readout_pair_2=readout_pair_2)


current_correlation_counts_corrected_post_selected = calculate_current_correlation_from_counts(counts_corrected_post_selected,population_average=None, 
                                                                       basis=basis, readout_pair_1=readout_pair_1, readout_pair_2=readout_pair_2)


plt.plot(times, current_correlation_counts_post_selected, 'o--')
plt.plot(times, current_correlation_counts_corrected_post_selected, 'o--')

In [ ]:
all_correlations = [current_correlation_counts, current_correlation_counts_corrected, current_correlation_counts_post_selected, current_correlation_counts_corrected_post_selected]
correlation_labels = ['Raw Counts', 'Corrected Counts', 'Post-Selected Counts', 'Corrected Post-Selected Counts']

plt.figure(figsize=(10,6))
for i in range(len(all_correlations)):
    plt.plot(times, all_correlations[i], 'o--', label=correlation_labels[i])
plt.xlabel('Time (ns)')
plt.ylabel('Current Correlation')
plt.ylim(-0.35, 0.35)
plt.legend()
plt.show()